In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import imp
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri

RANDOM_SEED = 0
numpy2ri.activate()
ro.r('set.seed({})'.format(RANDOM_SEED))

import fairtest.utils.log as fairtest_log
imp.reload(fairtest_log)
fairtest_log.set_params(filename='fairtest.log')

In [3]:
import fairtest.utils.prepare_data as prepare
from fairtest import DataSource
import fairtest.investigation as inv
import fairtest.testing as testing
import fairtest.discovery as discovery
import fairtest.error_profiling as error_profiling
import fairtest.modules.metrics as metrics
import fairtest.modules.metrics.correlation as correlation
import fairtest.modules.metrics.regression as regression
import fairtest.modules.metrics.binary_metrics as binary_metrics
import fairtest.modules.statistics.confidence_interval as intervals
import ast
import sklearn.preprocessing as preprocessing


dataname = 'movies'

# Testing Investigation on Movie Ratings
data = prepare.data_from_csv( '../data/recommender/recommendations.txt', sep='\\t')

# prepare labels
label_col = 'Types'
labeled_data = [ast.literal_eval(s) for s in data[label_col]]
for labels in labeled_data:
    assert len(labels) == 5
label_encoder = preprocessing.MultiLabelBinarizer()
labeled_data = label_encoder.fit_transform(labeled_data)
labels = label_encoder.classes_
df_labels = pd.DataFrame(labeled_data, columns=labels)
data = pd.concat([data.drop(label_col, axis=1), df_labels], axis=1)
labels = labels.tolist()

# prepare age
data['Age'] = map(lambda a: 10 if a == 1 
                       else 20 if a == 18 
                       else 30 if a == 25 
                       else 40 if a == 35 
                       else 50 if a == 45 or a == 50
                       else 60 if a == 56 else None, data['Age'])

data['Age_bin'] = map(lambda a: '<35' if a < 35 else '>=35', data['Age'])
data_source = DataSource(data)


#
# Test of associations on movie popularity
#
SENS = ['Gender', 'Age']
TARGET = 'Avg Movie Rating'
EXPL = []

test_ratings = testing.Testing(data_source, SENS, TARGET, EXPL, random_state=RANDOM_SEED,
                               to_drop=['RMSE', 'Avg Movie Age', 'Age_bin'] + labels)



#
# Test of associations on error
#
SENS = ['Gender', 'Age']
TARGET = 'RMSE'
EXPL = []
test_rmse = testing.Testing(data_source, SENS, TARGET, EXPL, random_state=RANDOM_SEED,
                            to_drop=['Avg Movie Rating', 'Avg Movie Age', 'Age_bin'] + labels)


#
# Discovery Investigation on Movie Types
#
TARGET = labels
SENS = ['Gender', 'Age_bin']
EXPL = []
discovery_types = discovery.Discovery(data_source, SENS, TARGET, EXPL, topk=15, random_state=RANDOM_SEED,
                                      to_drop=['Avg Movie Rating', 'Avg Movie Age', 'RMSE', 'Age'])
                                      

#
# Testing Investigation on Movie Age                                      
#
TARGET = 'Avg Movie Age'
SENS = ['Gender', 'Age']
EXPL = []                    
test_ages = testing.Testing(data_source, SENS, TARGET, EXPL, random_state=RANDOM_SEED,
                            to_drop=['Avg Movie Rating', 'RMSE', 'Age_bin'] + labels)

investigations = [test_ratings, test_rmse, test_ages, discovery_types]

INFO:root:Encoding Feature Gender
INFO:root:Encoding Feature Occupation
INFO:root:Encoding Feature Age_bin
INFO:root:Training Size 3020
INFO:root:Testing Sizes [3020]
INFO:root:New Testing Investigation
INFO:root:Target Feature: Target(names=['Avg Movie Rating'], arity=None)
INFO:root:Choosing metric CORR for feature Gender
INFO:root:Choosing metric CORR for feature Age
INFO:root:New Testing Investigation
INFO:root:Target Feature: Target(names=['RMSE'], arity=None)
INFO:root:Choosing metric CORR for feature Gender
INFO:root:Choosing metric CORR for feature Age
INFO:root:New Discovery Investigation
INFO:root:Target Feature: Target(names=[Action ... Western], arity=None)
INFO:root:Choosing metric REGRESSION for feature Age_bin
INFO:root:Choosing metric REGRESSION for feature Gender
INFO:root:New Testing Investigation
INFO:root:Target Feature: Target(names=['Avg Movie Age'], arity=None)
INFO:root:Choosing metric CORR for feature Gender
INFO:root:Choosing metric CORR for feature Age


In [4]:
import fairtest.modules.context_discovery.guided_tree as guided_tree
imp.reload(guided_tree)
inv.train(investigations, score_aggregation='avg', min_leaf_size=100, max_depth=5)

INFO:root:Begin training phase with protected feature Gender
INFO:root:Building a Guided Decision Tree
INFO:root:splitting on Age (score=0.0119413682903) with threshold 45.0 at pred []
INFO:root:splitting on Age (score=0.0120974782475) with threshold 25.0 at pred ['Age <= 45.0']
INFO:root:splitting on Occupation (score=0.01319515324) with threshold None at pred ['Age <= 45.0', 'Age > 25.0']
INFO:root:Begin training phase with protected feature Age
INFO:root:Building a Guided Decision Tree
INFO:root:splitting on Gender (score=0.114767969484) with threshold None at pred []
INFO:root:splitting on Occupation (score=0.0195100589938) with threshold None at pred ['Gender = 1']
INFO:root:Begin training phase with protected feature Gender
INFO:root:Building a Guided Decision Tree
INFO:root:splitting on Age (score=0.0561652949309) with threshold 25.0 at pred []
INFO:root:splitting on Age (score=0.0347857750395) with threshold 15.0 at pred ['Age <= 25.0']
INFO:root:splitting on Occupation (score=

In [5]:
import fairtest.modules.statistics.hypothesis_test as tests
import fairtest.modules.statistics.multiple_testing as multitest
import fairtest.modules.context_discovery.tree_parser as tree_parser
import fairtest.modules.statistics.confidence_interval as intervals
imp.reload(tests)
imp.reload(multitest)
imp.reload(tree_parser)
imp.reload(intervals)

inv.test(investigations, exact=True, prune_insignificant=True)

INFO:root:Parsing tree for sensitive feature Gender...
INFO:root:Parsed tree for sensitive feature Gender
INFO:root:Parsing tree for sensitive feature Age...
INFO:root:Parsed tree for sensitive feature Age
INFO:root:Parsing tree for sensitive feature Gender...
INFO:root:Parsed tree for sensitive feature Gender
INFO:root:Parsing tree for sensitive feature Age...
INFO:root:Parsed tree for sensitive feature Age
INFO:root:Parsing tree for sensitive feature Gender...
INFO:root:Parsed tree for sensitive feature Gender
INFO:root:Parsing tree for sensitive feature Age...
INFO:root:Parsed tree for sensitive feature Age
INFO:root:Parsing tree for sensitive feature Age_bin...
INFO:root:Parsed tree for sensitive feature Age_bin
INFO:root:Parsing tree for sensitive feature Gender...
INFO:root:Parsed tree for sensitive feature Gender
INFO:root:Begin testing phase
INFO:root:Testing 10 hypotheses
INFO:root:Computing stats for 6 contexts
INFO:root:Computing stats for context 0
INFO:root:Computing stats

In [9]:
import fairtest.modules.bug_report.report as rep
import fairtest.modules.bug_report.filter_rank as fr
imp.reload(rep)
imp.reload(fr)

output_dir = '../results'
#output_dir = None
inv.report(investigations, dataname, filter_conf=0.95, output_dir=output_dir, node_filter=fr.FILTER_BETTER_THAN_ANCESTORS)

INFO:root:Filtering and ranking 4 sub-contexts
INFO:root:0 statistically significant sub-contexts
INFO:root:0 sub-contexts printed
INFO:root:Filtering and ranking 6 sub-contexts
INFO:root:2 statistically significant sub-contexts
INFO:root:Size range: 846-2174
INFO:root:1 sub-contexts printed
INFO:root:Filtering and ranking 9 sub-contexts
INFO:root:1 statistically significant sub-contexts
INFO:root:Size range: 2353-2353
INFO:root:1 sub-contexts printed
INFO:root:Filtering and ranking 3 sub-contexts
INFO:root:1 statistically significant sub-contexts
INFO:root:Size range: 2174-2174
INFO:root:0 sub-contexts printed
INFO:root:Filtering and ranking 3 sub-contexts
INFO:root:0 statistically significant sub-contexts
INFO:root:0 sub-contexts printed
INFO:root:Filtering and ranking 3 sub-contexts
INFO:root:2 statistically significant sub-contexts
INFO:root:Size range: 846-2174
INFO:root:0 sub-contexts printed
INFO:root:Filtering and ranking 13 sub-contexts
INFO:root:1 statistically significant su

In [ ]:
# Testing Investigation on Movie Ratings
data = prepare.data_from_csv( '../data/recommender/recommendations.txt', sep='\\t')

# prepare age
data['Age'] = map(lambda a: 10 if a == 1 
                       else 20 if a == 18 
                       else 30 if a == 25 
                       else 40 if a == 35 
                       else 50 if a == 45 or a == 50
                       else 60 if a == 56 else None, data['Age'])

data['RMSE'] = ['low' if err < np.median(data['RMSE']) else 'high' for err in data['RMSE']]

data_source = DataSource(data)


#
# Test of associations on movie popularity conditioned on error
#
SENS = ['Gender', 'Age']
TARGET = 'Avg Movie Rating'
EXPL = ['RMSE']

test_ratings = testing.Testing(data_source, SENS, TARGET, EXPL, random_state=RANDOM_SEED,
                               to_drop=['Avg Movie Age', 'Types'])

In [ ]:
inv.train([test_ratings])

In [ ]:
inv.test([test_ratings])

In [ ]:
inv.report([test_ratings])

In [ ]:
output_dir = None
inv.report([test_ratings], dataname, output_dir=output_dir, filter_conf=0.95, node_filter=fr.FILTER_BETTER_THAN_ANCESTORS)